# U.S. Nuclear Investment and Policy Overview

## U.S. Nuclear Investment

By Abi Insani

### Introduction

In the early 2020s, there has been a reesurgence of nuclear energy-driven less by ideology but more by policy pragmatism. State governments have seen overlapping agenda: decarbonization push, energy security concerns, and grid modernization. As a result, state and federal actors have moved from debating whether "nuclear is" to negotiating "nuclear how", and this culminated in **Executive Order 14299 - Deploying Advanced Nuclear Reactor Technologies for National Security,** where President Donald Trump authorized the rapid deployment of advanced nuclear reactor.

This page highlights how U.S. States have building policies supporting nuclear power, namely from energy planning, siting, to workforce development.


### I. Data Sources and Structure



In [12]:
from IPython.display import HTML, display

display(
    HTML(r"""
<style>
:root{
  --border:#d0d7de;
  --subtle:#f6f8fa;
  --text:#1f2328;
  --muted:#59636e;
}
.kpi-wrap{
  max-width: 1080px;
  margin: 8px 0 0 0;
  font-family: -apple-system,BlinkMacSystemFont,"Segoe UI",Helvetica,Arial,sans-serif;
  color: var(--text);
}
.kpi-title{
  font-size: 28px;
  font-weight: 800;
  margin: 0 0 14px 0;
  padding-bottom: 10px;
  border-bottom: 1px solid var(--border);
}
.kpi-grid{
  display: grid;
  grid-template-columns: repeat(4, 1fr);
  gap: 18px;
}
.kpi-card{
  border: 1px solid var(--border);
  border-radius: 12px;
  background: #fff;
  padding: 18px 18px 16px;
  box-shadow: 0 1px 0 rgba(31,35,40,0.04);
}
.kpi-label{
  color: var(--muted);
  font-size: 14px;
  line-height: 1.25;
  margin-bottom: 10px;
}
.kpi-value{
  font-size: 34px;
  font-weight: 800;
  line-height: 1;
}
@media (max-width: 900px){
  .kpi-grid{ grid-template-columns: repeat(2, 1fr); }
}
@media (max-width: 520px){
  .kpi-grid{ grid-template-columns: 1fr; }
}
</style>

<div class="kpi-wrap">
  <div class="kpi-title">I. Data Sources and Structure</div>

  <div class="kpi-grid">
    <div class="kpi-card">
      <div class="kpi-label">States in investment dataset</div>
      <div class="kpi-value">50</div>
    </div>

    <div class="kpi-card">
      <div class="kpi-label">Policy mentions (INL/GAIN-classified)</div>
      <div class="kpi-value">85</div>
    </div>

    <div class="kpi-card">
      <div class="kpi-label">Unique states in policy mentions</div>
      <div class="kpi-value">38</div>
    </div>

    <div class="kpi-card">
      <div class="kpi-label">Distinct policy themes</div>
      <div class="kpi-value">10</div>
    </div>
  </div>
</div>
""")
)

### Datasets Used
- **U.S. Nuclear Investment by State (50 states)** — quantitative state-level fields (reactor units, planned units, restart activity, explicit federal support, private/state-reported figures, score, tier)
- **INL/GAIN State Nuclear Policy Mentions (Feb 2025 framework)** — classified examples and “see also” mentions by theme/subtheme

### Preparations Steps

1. Import data from Nuclear Energy Institute (NEI) and Idaho National Lab (INL)/Gateway for Accelerated Innovation in Nuclear (GAIN) and convert to CSV.
2. Collect key qualitative policy data and count based on policy instrument from INL/GAIN classification.
3. Create dashboard summaries: Policy theme coverage, state classification lists, and investment score.
4. Upload dashboard to github.


## II. Nuclear Investments and Scoring by State

In [ ]:
import pandas as pd
import numpy as np

from IPython.display import HTML, display
import plotly.graph_objects as go
import plotly.express as px

# Load your uploaded CSVs
inv = pd.read_csv("us_nuclear_investment_by_state_asof_2026-02-25.csv", sep=";")
policy = pd.read_csv("inl_nuclear_policy_mentions_classified.csv", sep=";")

# Clean types (safe casts)
int_cols = [
    "fips",
    "operating_reactor_units",
    "planned_new_reactor_units",
    "restart_existing_reactor_units",
]
for c in int_cols:
    inv[c] = pd.to_numeric(inv[c], errors="coerce").fillna(0).astype(int)

float_cols = [
    "restart_capacity_mw",
    "explicit_federal_support_usd_m",
    "state_reported_nuclear_investment_usd_m",
    "project_reported_private_investment_usd_m",
    "investment_score",
]
for c in float_cols:
    inv[c] = pd.to_numeric(inv[c], errors="coerce").fillna(0.0)

policy["theme"] = policy["theme"].fillna("").astype(str)
policy["state"] = policy["state"].fillna("").astype(str)


# Cell 2
display(HTML('<div class="section-title">III. Data Visualization and Analysis</div>'))
display(
    HTML(
        '<div class="fig-title">Figure 1 — Top Federal Support by State (USD million)</div>'
    )
)

top_support = (
    inv[inv["explicit_federal_support_usd_m"] > 0]
    .sort_values("explicit_federal_support_usd_m", ascending=False)
    .head(12)
)

# Plotly bar chart
fig1 = go.Figure(
    data=[
        go.Bar(
            x=top_support["abbr"],
            y=top_support["explicit_federal_support_usd_m"],
            text=[f"${v:,.0f}M" for v in top_support["explicit_federal_support_usd_m"]],
            textposition="inside",
            marker=dict(
                color=top_support["explicit_federal_support_usd_m"],
                colorscale="Viridis",
                showscale=False,
            ),
        )
    ]
)
fig1.update_layout(
    title="Top states by explicit federal support (USD million) in 2025",
    xaxis_title="State",
    yaxis_title="USD million",
    margin=dict(l=50, r=20, t=60, b=50),
    height=420,
)

# Table HTML
tbl1 = (
    top_support[["state", "explicit_federal_support_usd_m"]]
    .rename(
        columns={
            "state": "State",
            "explicit_federal_support_usd_m": "Federal Support (USD million)",
        }
    )
    .to_html(index=False, border=0)
)

# Side-by-side layout
display(HTML('<div class="grid-2">'))
display(HTML('<div class="card">'))
fig1.show()
display(HTML("</div>"))

display(
    HTML(f"""
<div class="card">
  <h4>Top support table</h4>
  {tbl1}
  <div class="note">Only explicitly stated federal support amounts are included in this field.</div>
</div>
</div>
""")
)

# Cell 3
display(
    HTML('<div class="fig-title">Figure 2 — Operating Reactor Units (Top States)</div>')
)

top_ops = (
    inv.sort_values(["operating_reactor_units", "investment_score"], ascending=False)
    .head(10)
    .copy()
)

# Plotly horizontal bar
data2 = top_ops.sort_values("operating_reactor_units", ascending=True)
fig2 = go.Figure(
    data=[
        go.Bar(
            x=data2["operating_reactor_units"],
            y=data2["state"],
            orientation="h",
            text=data2["operating_reactor_units"],
            textposition="inside",
            marker=dict(
                color=data2["operating_reactor_units"],
                colorscale="Turbo",  # very colorful modern gradient
                showscale=False,
            ),
        )
    ]
)

fig2.update_layout(
    title="Top states by operating reactor units",
    xaxis_title="Operating reactor units",
    template="plotly_white",
    margin=dict(l=120, r=20, t=60, b=50),
    height=420,
)


tbl2 = (
    top_ops[
        [
            "state",
            "operating_reactor_units",
            "planned_new_reactor_units",
            "investment_score",
        ]
    ]
    .rename(
        columns={
            "state": "State",
            "operating_reactor_units": "Operating Reactor Units",
            "planned_new_reactor_units": "Planned New Reactor Units",
            "investment_score": "Investment Score",
        }
    )
    .to_html(index=False, border=0)
)

display(HTML('<div class="grid-2">'))
display(HTML('<div class="card">'))
fig2.show()
display(HTML("</div>"))

display(
    HTML(f"""
<div class="card">
  <h4>Operating fleet preview</h4>
  {tbl2}
</div>
</div>
""")
)

# Cell 4
display(
    HTML('<div class="fig-title">Figure 3 — Investment Score by State (All 50)</div>')
)

score_sorted = (
    inv[inv["investment_score"] > 0]  # 🔹 remove zero-score states
    .sort_values("investment_score", ascending=False)
    .copy()
)

fig3 = go.Figure(
    data=[
        go.Bar(
            x=score_sorted["abbr"],
            y=score_sorted["investment_score"],
            text=[f"{v:.1f}" for v in score_sorted["investment_score"]],
            textposition="outside",
            marker=dict(
                color=score_sorted["investment_score"],  # color by value
                colorscale="Turbo",  # very colorful gradient
                showscale=False,
            ),
            hovertemplate=(
                "<b>%{x}</b><br>"
                "Investment Score: %{y:.2f}<br>"
                "%{hovertext}<extra></extra>"
            ),
            hovertext=[
                f"{s} • {t}"
                for s, t in zip(
                    score_sorted["state"],
                    score_sorted.get("tier", [""] * len(score_sorted)),
                )
            ],
        )
    ]
)

fig3.update_layout(
    title="Investment Score Ranking",
    xaxis_title="State",
    yaxis_title="Investment Score",
    xaxis_tickangle=-45,
    template="plotly_white",
    margin=dict(l=50, r=20, t=60, b=120),
    height=500,
)

display(HTML('<div class="card">'))
fig3.show()
display(HTML("</div>"))

# Cell 5
display(
    HTML(
        '<div class="fig-title">Figure 4 — Scatter: Operating Units vs Explicit Federal Support</div>'
    )
)

# Create a label column
inv["label"] = inv.apply(
    lambda row: row["abbr"] if row["explicit_federal_support_usd_m"] > 0 else "", axis=1
)

fig4 = px.scatter(
    inv,
    x="operating_reactor_units",
    y="explicit_federal_support_usd_m",
    size=inv["planned_new_reactor_units"] + inv["restart_existing_reactor_units"] + 1,
    color="investment_score",
    color_continuous_scale="Turbo",
    hover_data=["state", "investment_score", "tier"],
    text="label",
)

fig4.update_traces(textposition="top center", marker=dict(opacity=0.8))

fig4.update_layout(
    template="plotly_white",
    title="Operating Units vs Explicit Federal Support",
    xaxis_title="Operating reactor units",
    yaxis_title="Explicit federal support (USD million)",
    height=450,
)

fig4.show()


# If you want to hide zero-score states on the map, uncomment the filter line
df_map = inv.copy()
# df_map = df_map[df_map["investment_score"] > 0]

fig_map = px.choropleth(
    df_map,
    locations="abbr",  # state abbreviations (e.g., CA, NY)
    locationmode="USA-states",
    color="investment_score",
    scope="usa",
    color_continuous_scale="Turbo",  # colorful gradient
    hover_name="state",
    hover_data={
        "abbr": True,
        "operating_reactor_units": True,
        "planned_new_reactor_units": True,
        "restart_existing_reactor_units": True,
        "explicit_federal_support_usd_m": True,
        "investment_score": True,
    },
    labels={
        "operating_reactor_units": "Operating Units",
        "planned_new_reactor_units": "Planned New Units",
        "restart_existing_reactor_units": "Restart Units",
        "explicit_federal_support_usd_m": "Federal Support (USD million)",
        "investment_score": "Investment Score",
    },
    title="U.S. Nuclear Investment Score by State",
)

fig_map.update_layout(
    template="plotly_white", height=600, margin=dict(l=10, r=10, t=60, b=10)
)

fig_map.show()

State,Federal Support (USD million)
Michigan,1920.0
Tennessee,1300.0
Kentucky,900.0
Ohio,900.0


State,Operating Reactor Units,Planned New Reactor Units,Investment Score
Illinois,11,0,11.0
Pennsylvania,8,0,8.0
South Carolina,7,0,7.0
Alabama,5,0,5.0
North Carolina,5,0,5.0
Texas,4,4,16.0
Tennessee,4,1,9.6
Florida,4,0,4.0
Georgia,4,0,4.0
New York,4,0,4.0


### Investment Score Calculations

The **Investment Score** is a calculated by weighted-score between *Operating, Planned, and Explicit Federal Support*.
The formula is:

Investment Score =  1 × Operating Reactor Units + 2 × Planned New Reactor Units + (Federal Funding (USD Millions) / 500)




### III. Nuclear Policies by U.S. States

In [ ]:
display(
    HTML('<div class="section-title">IV. INL/GAIN Policy Classification Summary</div>')
)

# Clean theme/state
policy_clean = policy.copy()
policy_clean["theme"] = policy_clean["theme"].fillna("").astype(str).str.strip()
policy_clean["state"] = policy_clean["state"].fillna("").astype(str).str.strip()

policy_clean = policy_clean[
    (policy_clean["theme"] != "") & (policy_clean["state"] != "")
].copy()

# Theme summary
theme_summary = (
    policy_clean.groupby("theme")
    .agg(total_mentions=("state", "size"), unique_states=("state", "nunique"))
    .reset_index()
    .sort_values(["unique_states", "total_mentions"], ascending=False)
)

theme_summary.head()

import plotly.graph_objects as go

theme_rename = {
    "STATE ENERGY PLANNING": "Planning & Standards",
    "FEASIBILITY STUDIES AND WORKING GROUPS": "Feasibility / Task Forces",
    "MORATORIUM REPEALS/EXEMPTIONS": "Moratorium Reform",
    "MARKET INCENTIVES": "Market Incentives",
    "REGULATORY": "Regulatory Instruments",
    "FOSSIL-FUEL TRANSITION": "Fossil Transition",
    "SUPPLY CHAIN/CLEAN ENERGY MANUFACTURING": "Supply Chain/Industrial Policy",
    "WORKFORCE DEVELOPMENT": "Workforce Development",
    "SITING": "Siting & Permitting",
    "ESTABLISHMENT OF AUTHORITIES": "Institutional Framework",
}

# Figure 5: Theme Coverage Bar Chart

TOP_N = 10
t = theme_summary.head(10).copy()
t["theme_display"] = t["theme"].replace(theme_rename)

fig5 = go.Figure()

# Unique states
fig5.add_trace(
    go.Bar(
        x=t["theme_display"],
        y=t["unique_states"],
        name="Unique states",
        marker=dict(
            color="#1f77b4"  # solid blue
        ),
        hovertemplate="<b>%{x}</b><br>Unique states: %{y}<extra></extra>",
    )
)

# Total mentions
fig5.add_trace(
    go.Bar(
        x=t["theme_display"],
        y=t["total_mentions"],
        name="Total mentions",
        marker=dict(
            color="#ff7f0e"  # solid orange
        ),
        hovertemplate="<b>%{x}</b><br>Total mentions: %{y}<extra></extra>",
    )
)

fig5.update_layout(
    title=f"INL/GAIN Policy Theme Coverage (Top {TOP_N})",
    barmode="group",
    template="plotly_white",
    height=480,
    margin=dict(l=50, r=20, t=70, b=160),
    xaxis=dict(title="", tickangle=-35),
    yaxis=dict(title="Count"),
)

display(HTML('<div class="card">'))
fig5.show()
display(HTML("</div>"))

# Figure 6: State x Theme Heatmap


# 1) Get the TOP themes in their original/raw form (these match the pivot columns)
top_themes_raw = theme_summary.head(10)["theme"].tolist()

# 2) Make display labels (these are ONLY for axis labels)
top_themes_display = [theme_rename.get(t, t) for t in top_themes_raw]

# 3) Filter to top themes + create binary flag
policy_top = policy_clean[policy_clean["theme"].isin(top_themes_raw)].copy()
policy_top["flag"] = 1

# 4) Pivot to binary matrix: state x theme (columns are RAW theme names)
heat = policy_top.pivot_table(
    index="state", columns="theme", values="flag", aggfunc="max", fill_value=0
)

# 5) Sort states by coverage across RAW theme columns
heat["coverage"] = heat[top_themes_raw].sum(axis=1)
heat = heat.sort_values("coverage", ascending=False).drop(columns=["coverage"])

# 6) Build arrays
states = heat.index.tolist()
Z = heat[top_themes_raw].to_numpy()  # ✅ IMPORTANT: raw names here

# 7) Binary colors (0/1 only)
fig6 = px.imshow(
    Z,
    x=top_themes_display,  # ✅ Pretty labels only
    y=states,
    aspect="auto",
    zmin=0,
    zmax=1,
    color_continuous_scale=[
        (0.0, "#f6f8fa"),
        (0.4999, "#f6f8fa"),
        (0.5, "#0969da"),
        (1.0, "#0969da"),
    ],
)

fig6.update_layout(
    title="State × Policy Instrument Heatmap (Binary, Top Themes Only)",
    template="plotly_white",
    height=650,
    margin=dict(l=160, r=20, t=70, b=160),
    xaxis=dict(tickangle=-35, title=""),
    yaxis=dict(title=""),
    coloraxis_showscale=False,
)

fig6.update_traces(
    hovertemplate="State: %{y}<br>Instrument: %{x}<br>Mentioned: %{z}<extra></extra>"
)

display(HTML('<div class="card">'))
fig6.show()